This is notebook gives a quick overview of this WaveNet implementation, i.e. creating the model and the data set, training the model and generating samples from it.

In [ ]:
# !python -m pip install tensorflow
# !pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
# !pip uninstall torch torchvision torchaudio


In [32]:
# checking which conda environment this kernel is using
import sys
print(sys.executable)

C:\Users\Ethan\anaconda3\envs\pythonProject\python.exe


In [1]:
import numpy as np
import torch
from wavenet_model import *
from audio_data import WavenetDataset #WavenetDataset() uses librosa
from wavenet_training import *
from model_logging import *
import tensorflow as tf

## Model
This is an implementation of WaveNet as it was described in the original paper (https://arxiv.org/abs/1609.03499). Each layer looks like this:

```
            |----------------------------------------|      *residual*
            |                                        |
            |    |-- conv -- tanh --|                |
 -> dilate -|----|                  * ----|-- 1x1 -- + -->  *input*
                 |-- conv -- sigm --|     |
                                         1x1
                                          |
 ---------------------------------------> + ------------->  *skip*
```

Each layer dilates the input by a factor of two. After each block the dilation is reset and start from one. You can define the number of layers in each block (``layers``) and the number of blocks (``blocks``). The blocks are followed by two 1x1 convolutions and a softmax output function.
Because of the dilation operation, the independent output for multiple successive samples can be calculated efficiently. With ``output_length``, you can define the number these outputs. Empirically, it seems that a large number of skip channels is required.

In [2]:
# initialize cuda option
dtype = torch.FloatTensor # data type
ltype = torch.LongTensor # label type

use_cuda = torch.cuda.is_available()

if use_cuda:
    print('use gpu')
    dtype = torch.cuda.FloatTensor
    # dtype=torch.tensor(data=None,device="cuda")
    ltype = torch.cuda.LongTensor
else:
    print('not using gpu')

use gpu


In [31]:
model = WaveNetModel(layers=10,
                     blocks=3,
                     dilation_channels=24,
                     residual_channels=32,
                     skip_channels=1024,
                     end_channels=256, 
                     output_length=16, # is this how many predictions we make?
                     dtype=dtype, 
                     bias=True)
model.to("cuda")
# model = load_latest_model_from('snapshots', use_cuda=use_cuda)

print('model: ', model)
print('receptive field: ', model.receptive_field)
print('parameter count: ', model.parameter_count())
print('output length:   ', model.output_length)

model:  WaveNetModel(
  (filter_convs): ModuleList(
    (0-29): 30 x Conv1d(24, 12, kernel_size=(2,), stride=(1,))
  )
  (gate_convs): ModuleList(
    (0-29): 30 x Conv1d(24, 12, kernel_size=(2,), stride=(1,))
  )
  (residual_convs): ModuleList(
    (0-29): 30 x Conv1d(12, 24, kernel_size=(1,), stride=(1,))
  )
  (skip_convs): ModuleList(
    (0-29): 30 x Conv1d(12, 256, kernel_size=(1,), stride=(1,))
  )
  (start_conv): Conv1d(256, 24, kernel_size=(1,), stride=(1,))
  (end_conv_1): Conv1d(256, 256, kernel_size=(1,), stride=(1,))
  (end_conv_2): Conv1d(256, 256, kernel_size=(1,), stride=(1,))
)
receptive field:  3070
parameter count:  282232
output length:    16


## Data Set
To create the data set, you have to specify a path to a data set file. If this file already exists it will be used, if not it will be generated. If you want to generate the data set file (a ``.npz`` file), you have to specify the directory (``file_location``) in which all the audio files you want to use are located. The attribute ``target_length`` specifies the number of successive samples are used as a target and corresponds to the output length of the model. The ``item_length`` defines the number of samples in each item of the dataset and should always be ``model.receptive_field + model.output_length - 1``.

```
          |----receptive_field----|
                                |--output_length--|
example:  | | | | | | | | | | | | | | | | | | | | |
target:                           | | | | | | | | | |  
```
To create a test set, you should define a ``test_stride``. Then each ``test_stride``th item will be assigned to the test set.

In [23]:
data = WavenetDataset(dataset_file='train_samples/dataset.npz',
                      item_length=model.receptive_field + model.output_length - 1,
                      target_length=model.output_length,
                      file_location='train_samples',
                      test_stride=1000000000000)
print('the dataset has ' + str(len(data)) + ' items')
print(f'{data[100][0].shape}')

one hot input
the dataset has 62308 items
torch.Size([256, 3085])


## Training and Logging
This implementation supports logging with TensorBoard (you need to have TensorFlow installed). You can even generate audio samples from the current snapshot of the model during training. This will happen in a background thread on the cpu, so it will not interfere with the actual training but will be rather slow. If you don't have TensorFlow, you can use the standard logger that will print out to the console.
The trainer uses Adam as default optimizer.

In [5]:
print(f"TensorFlow Version: {tf.__version__}")
def generate_and_log_samples(step):
    sample_length=32000
    gen_model = load_latest_model_from('snapshots', use_cuda=False) # (Apparently) generating with Fast Wavenet is faster on cpu
    print("start generating...")
    samples = generate_audio(gen_model,
                             length=sample_length,
                             temperatures=[0.5])
    tf_samples = tf.convert_to_tensor(samples, dtype=tf.float32)
    logger.audio_summary('temperature_0.5', tf_samples, step, sr=16000)

    samples = generate_audio(gen_model,
                             length=sample_length,
                             temperatures=[1.])
    tf_samples = tf.convert_to_tensor(samples, dtype=tf.float32)
    logger.audio_summary('temperature_1.0', tf_samples, step, sr=16000)
    print("audio clips generated")


logger = TensorboardLogger(log_interval=200,
                           validation_interval=400,
                           generate_interval=1000,
                           generate_function=generate_and_log_samples,
                           log_dir="logs/model")

# logger = Logger(log_interval=200,
#                 validation_interval=400,
#                 generate_interval=1000)

TensorFlow Version: 2.19.0
2.19.0




In [6]:
trainer = WavenetTrainer(model=model,
                         dataset=data,
                         lr=0.001,
                         snapshot_path='snapshots',
                         snapshot_name='chaconne_model',
                         snapshot_interval=1000,
                         logger=logger,
                         dtype=dtype,
                         ltype=ltype)



# we should just train on a tiny dataset first to speed up debugging
print('start training...')
trainer.train(batch_size=4, epochs=2)

start training...
epoch 0
On batch number: 0 of 15334
one training step does take approximately 0.44217905759811404 seconds)
On batch number: 250 of 15334
On batch number: 500 of 15334
On batch number: 750 of 15334


Exception in thread Thread-4 (generate_and_log_samples):
Traceback (most recent call last):
  File "C:\Users\Ethan\anaconda3\envs\pythonProject\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "C:\Users\Ethan\anaconda3\envs\pythonProject\lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "C:\Users\Ethan\anaconda3\envs\pythonProject\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Ethan\AppData\Local\Temp\ipykernel_6908\2911061226.py", line 4, in generate_and_log_samples
  File "C:\Users\Ethan\Documents\MATH 479 (Math Modelling)\WaveNet-Implementation\wavenet_model.py", line 344, in load_latest_model_from
    model = load_to_cpu(newest_file)
  File "C:\Users\Ethan\Documents\MATH 479 (Math Modelling)\WaveNet-Implementation\wavenet_model.py", line 350, in load_to_cpu
    model = torch.load(path, map_location=lambda storage, loc: storage)
  File "C:\Users\Ethan\an

load model snapshots/chaconne_model_2025-05-03_20-23-24
On batch number: 1000 of 15334
On batch number: 1250 of 15334
On batch number: 1500 of 15334
On batch number: 1750 of 15334


Exception in thread Thread-5 (generate_and_log_samples):
Traceback (most recent call last):
  File "C:\Users\Ethan\anaconda3\envs\pythonProject\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "C:\Users\Ethan\anaconda3\envs\pythonProject\lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "C:\Users\Ethan\anaconda3\envs\pythonProject\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Ethan\AppData\Local\Temp\ipykernel_6908\2911061226.py", line 4, in generate_and_log_samples
  File "C:\Users\Ethan\Documents\MATH 479 (Math Modelling)\WaveNet-Implementation\wavenet_model.py", line 344, in load_latest_model_from
    model = load_to_cpu(newest_file)
  File "C:\Users\Ethan\Documents\MATH 479 (Math Modelling)\WaveNet-Implementation\wavenet_model.py", line 350, in load_to_cpu
    model = torch.load(path, map_location=lambda storage, loc: storage)
  File "C:\Users\Ethan\an

load model snapshots/chaconne_model_2025-05-03_20-34-57
On batch number: 2000 of 15334
On batch number: 2250 of 15334
On batch number: 2500 of 15334
On batch number: 2750 of 15334


...

load model snapshots/chaconne_model_2025-05-04_01-51-38
On batch number: 28000 of 15334
On batch number: 28250 of 15334
On batch number: 28500 of 15334
On batch number: 28750 of 15334


Exception in thread Thread-32 (generate_and_log_samples):
Traceback (most recent call last):
  File "C:\Users\Ethan\anaconda3\envs\pythonProject\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "C:\Users\Ethan\anaconda3\envs\pythonProject\lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "C:\Users\Ethan\anaconda3\envs\pythonProject\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Ethan\AppData\Local\Temp\ipykernel_6908\2911061226.py", line 4, in generate_and_log_samples
  File "C:\Users\Ethan\Documents\MATH 479 (Math Modelling)\WaveNet-Implementation\wavenet_model.py", line 344, in load_latest_model_from
    model = load_to_cpu(newest_file)
  File "C:\Users\Ethan\Documents\MATH 479 (Math Modelling)\WaveNet-Implementation\wavenet_model.py", line 350, in load_to_cpu
    model = torch.load(path, map_location=lambda storage, loc: storage)
  File "C:\Users\Ethan\a

load model snapshots/chaconne_model_2025-05-04_02-03-50
On batch number: 29000 of 15334
On batch number: 29250 of 15334
On batch number: 29500 of 15334
On batch number: 29750 of 15334


Exception in thread Thread-33 (generate_and_log_samples):
Traceback (most recent call last):
  File "C:\Users\Ethan\anaconda3\envs\pythonProject\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "C:\Users\Ethan\anaconda3\envs\pythonProject\lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "C:\Users\Ethan\anaconda3\envs\pythonProject\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Ethan\AppData\Local\Temp\ipykernel_6908\2911061226.py", line 4, in generate_and_log_samples
  File "C:\Users\Ethan\Documents\MATH 479 (Math Modelling)\WaveNet-Implementation\wavenet_model.py", line 344, in load_latest_model_from
    model = load_to_cpu(newest_file)
  File "C:\Users\Ethan\Documents\MATH 479 (Math Modelling)\WaveNet-Implementation\wavenet_model.py", line 350, in load_to_cpu
    model = torch.load(path, map_location=lambda storage, loc: storage)
  File "C:\Users\Ethan\a

load model snapshots/chaconne_model_2025-05-04_02-14-36
On batch number: 30000 of 15334
On batch number: 30250 of 15334
On batch number: 30500 of 15334


## Generating
This model has the Fast Wavenet Generation Algorithm (https://arxiv.org/abs/1611.09482) implemented. This might run faster on the cpu. You can give some starting data (of at least the length of receptive field) or let the model generate from zero. In my experience, a temperature between 0.5 and 1.0 yields the best results, but this may depend on the data set.

In [15]:
start_data = data[25000][0] # use start data from the data set
start_data = torch.max(start_data, 0)[1] # convert one hot vectors to integers
print(start_data)
def prog_callback(step, total_steps):
    print(str(100 * step // total_steps) + "% generated")
model = load_latest_model_from('snapshots', use_cuda=True)
generated = model.generate_fast(num_samples=160000,
                                 first_samples=start_data,
                                 progress_callback=prog_callback,
                                 progress_interval=1000,
                                 temperature=1.0,
                                 regularize=0.)

tensor([0, 0, 0,  ..., 0, 0, 0])
load model snapshots/.ipynb_checkpoints


PermissionError: [Errno 13] Permission denied: 'snapshots/.ipynb_checkpoints'

In [ ]:
import IPython.display as ipd

ipd.Audio(generated, rate=16000)